<a href="https://colab.research.google.com/github/carolinaenriqz/Optimization-Algorithms-for-Uncertain-Knapsack-Problem/blob/main/ExperimentosComputacionalesRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and Reading Instances

In [1]:
!pip -q install XlsxWriter
!pip install mip

!git clone --depth 1 https://github.com/likr/kplib.git

import os
import pandas as pd
from IPython.display import display
from google.colab import files
import numpy as np
import sys, subprocess, time
from mip import Model, CONTINUOUS, MAXIMIZE, CBC, OptimizationStatus
from scipy.stats import t as student_t


BASE_DIR = "/content/kplib/00Uncorrelated"
n_values = ["n00050", "n00100", "n00200", "n00500"]
instance_type = "R01000"

instances = []
for n in n_values:
    path = os.path.join(BASE_DIR, n, instance_type)
    if not os.path.isdir(path):
        print(f"Path not found: {path}")
        continue

    files_list = sorted([f for f in os.listdir(path) if f.lower().endswith('.kp')])[:20]

    for file in files_list:
        file_path = os.path.join(path, file)
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            num_items = int(lines[1].strip())
            capacity = int(lines[2].strip())
            items = [tuple(map(int, line.strip().split()))
                     for line in lines[4:] if line.strip()]

            if len(items) != num_items:
                print(f"Warning: in {file} {num_items} items were expected but {len(items)} were read")
                continue

            values, weights = zip(*items)
            instances.append({
                "n": num_items,
                "Capacity": capacity,
                "Utilities": list(values),
                "Weights": list(weights),
                "File": file
            })
        except Exception as e:
            print(f"Error reading {file}: {e}")

df_kplib = pd.DataFrame(instances)
print(f"Loaded instances: {df_kplib.shape}")
df_kplib = df_kplib[["File", "n", "Capacity", "Utilities", "Weights"]]
display(df_kplib.head(10))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.1/484.1 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 95.2 MB/s eta 0:00:00
  Created wheel for cffi: filename=cffi-1.15.0-cp312-cp312-linux_x86_64.whl size=400939 sha256=ba8ea202f96caab48c8623ae5d802fa12531240fc9c4eef63ccd3b83a79aada1
  Stored in directory: /root/.cache/pip/wheels/b9/d6/15/0950847bf7d74ea5f0380b8b23a1d81b45bdf48488b4b8237a
Successfully built cffi
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.2 requires cffi>=1.17.0, but you have cffi 1.15.0 which is incompatible.
rpy2 3.5.17 requires 

,File,n,Capacity,Utilities,Weights
0,s000.kp,50,14778,"[845, 758, 421, 259, 512, 405, 784, 304, 477, ...","[804, 448, 81, 321, 508, 933, 110, 552, 707, 5..."
1,s001.kp,50,13598,"[135, 848, 764, 256, 496, 450, 652, 789, 94, 2...","[304, 588, 883, 847, 506, 590, 35, 243, 798, 4..."
2,s002.kp,50,13810,"[957, 948, 57, 85, 836, 736, 670, 309, 606, 60...","[491, 925, 501, 832, 354, 883, 900, 462, 568, ..."
3,s003.kp,50,13596,"[238, 545, 370, 604, 626, 66, 14, 838, 260, 23...","[682, 929, 857, 991, 672, 164, 861, 965, 905, ..."
4,s004.kp,50,11618,"[237, 104, 397, 155, 67, 402, 918, 801, 766, 2...","[281, 535, 472, 343, 998, 196, 413, 203, 633, ..."
5,s005.kp,50,11922,"[623, 742, 796, 943, 740, 923, 30, 466, 944, 6...","[338, 310, 819, 481, 316, 482, 705, 58, 976, 2..."
6,s006.kp,50,12044,"[794, 822, 486, 262, 1, 663, 471, 760, 374, 77...","[881, 526, 117, 663, 313, 197, 484, 139, 210, ..."
7,s007.kp,50,12828,"[324, 151, 651, 73, 536, 366, 58, 508, 38, 434...","[981, 119, 419, 758, 152, 489, 40, 669, 765, 5..."
8,s008.kp,50,10985,"[227, 963, 127, 705, 86, 248, 1000, 210, 642, ...","[703, 950, 844, 504, 198, 151, 529, 510, 72, 9..."
9,s009.kp,50,11487,"[464, 374, 139, 867, 7, 503, 899, 81, 555, 617...","[84, 540, 18, 85, 497, 921, 421, 399, 639, 94,..."


# RO Solver Function (`solve_bs`)

In [ ]:
def solve_nominal_fractional_mip(c_bar, weights, capacity, time_limit=3600, solver_name=CBC):
    c_bar = np.asarray(c_bar, float)
    weights = np.asarray(weights, float)
    n = c_bar.size

    m = Model(sense=MAXIMIZE, solver_name=solver_name)
    m.max_seconds = time_limit
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]

    m.objective = sum(c_bar[j]*x[j] for j in range(n))
    m += (sum(weights[j]*x[j] for j in range(n)) <= capacity), "cap"

    t0 = time.perf_counter(); status = m.optimize(); t1 = time.perf_counter()
    obj = m.objective_value if status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE) else np.nan
    return float(obj), status, t1-t0


def solve_knapsack_bs_mip(c_bar, c_hat, weights, capacity, gamma, time_limit=3600, solver_name=CBC):
    c_bar = np.asarray(c_bar, float)
    c_hat = np.asarray(c_hat, float)
    weights = np.asarray(weights, float)
    n = c_bar.size
    assert c_hat.size == n and weights.size == n, "Inconsistent dimensions"

    m = Model(sense=MAXIMIZE, solver_name=solver_name)
    m.max_seconds = time_limit

    # Variables
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]
    z = m.add_var(var_type=CONTINUOUS, lb=0.0, name="z")
    p = [m.add_var(var_type=CONTINUOUS, lb=0.0, name=f"p_{j}") for j in range(n)]

    # Robust objective
    m.objective = sum(c_bar[j]*x[j] for j in range(n)) - gamma*z - sum(p[j] for j in range(n))

    # Capacity
    m += (sum(weights[j]*x[j] for j in range(n)) <= capacity), "cap"

    # Robustness: d_j x_j - z - p_j <= 0
    for j in range(n):
        m += (c_hat[j]*x[j] - z - p[j] <= 0), f"rob_{j}"

    t0 = time.perf_counter()
    status = m.optimize()
    t1 = time.perf_counter()
    ok = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
    obj = float(m.objective_value) if ok else np.nan

    # Extra: nominal evaluation of the solution x* (without penalties)
    if ok:
        sol_x = np.array([var.x for var in x], dtype=float)
        eval_cx = float(np.dot(c_bar, sol_x))
        sol_z = float(z.x)
        sol_p = np.array([var.x for var in p], dtype=float)
    else:
        sol_x = None
        eval_cx = np.nan
        sol_z = np.nan
        sol_p = None

    return {
        "ok": ok,
        "status": str(status),
        "objective": obj,         # robust: c·x - gamma z - sum p
        "eval_cx": eval_cx,       # nominal: c·x
        "x": sol_x,
        "z": sol_z,
        "p": sol_p,
        "time_s": round(t1 - t0, 6),
        "n_vars": m.num_cols,
        "n_constrs": m.num_rows,
        "model": m,
    }



=== n = 50 | 20 instancias | pct_desv=0.1 ===

=== n = 100 | 20 instancias | pct_desv=0.1 ===

=== n = 200 | 20 instancias | pct_desv=0.1 ===

=== n = 500 | 20 instancias | pct_desv=0.1 ===

=== n = 50 | 20 instancias | pct_desv=0.5 ===

=== n = 100 | 20 instancias | pct_desv=0.5 ===

=== n = 200 | 20 instancias | pct_desv=0.5 ===

=== n = 500 | 20 instancias | pct_desv=0.5 ===

=== n = 50 | 20 instancias | pct_desv=1 ===

=== n = 100 | 20 instancias | pct_desv=1 ===

=== n = 200 | 20 instancias | pct_desv=1 ===

=== n = 500 | 20 instancias | pct_desv=1 ===

=== n = 50 | 20 instancias | pct_desv=1.5 ===

=== n = 100 | 20 instancias | pct_desv=1.5 ===

=== n = 200 | 20 instancias | pct_desv=1.5 ===

=== n = 500 | 20 instancias | pct_desv=1.5 ===

=== n = 50 | 20 instancias | pct_desv=2 ===

=== n = 100 | 20 instancias | pct_desv=2 ===

=== n = 200 | 20 instancias | pct_desv=2 ===

=== n = 500 | 20 instancias | pct_desv=2 ===

=== Resumen (primeras filas) ===
   Archivo   n  Capacidad  

# Processing BS Results

In [ ]:
def total_solver_bs_mip(
    df_kplib,
    pct_deviation_list=(0.1, 0.5, 1.0, 1.5, 2.0),
    gamma_pcts=(0.00, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00),
    sizes=(50, 100, 200, 500),
    k=20,
    time_limit=3600,
    check_nominal=True,
):
    rows = []

    for pct_deviation in pct_deviation_list:
        for n_target in sizes:
            df_sub = df_kplib[df_kplib["n"] == n_target].head(k)
            print(f"\n=== n = {n_target} | {len(df_sub)} instances | pct_dev={pct_deviation} ===")
            for idx, row in df_sub.iterrows():
                cbar = np.array(row["Utilities"], float)
                w    = np.array(row["Weights"], float)
                cap  = float(row["Capacity"])
                arc  = row["File"] if "File" in df_kplib.columns else f"idx_{idx}"
                n    = cbar.size
                assert w.size == n, f"Dim mismatch in {arc}"

                # d_j = pct * |c̄_j|
                chat = np.abs(pct_deviation * cbar)

                # Nominal check (Γ=0) vs nominal fractional LP (with MIP as well)
                if check_nominal:
                    f_nom, st_nom, _ = solve_nominal_fractional_mip(cbar, w, cap, time_limit=time_limit)
                else:
                    f_nom = np.nan

                for pct in gamma_pcts:
                    gamma = int(round(pct * n))
                    sol = solve_knapsack_bs_mip(cbar, chat, w, cap, gamma, time_limit=time_limit)

                    if sol["ok"]:
                        if check_nominal and gamma == 0 and np.isfinite(f_nom):
                            diff = abs(sol["objective"] - f_nom)
                            if diff > 1e-6:
                                print(f"[Warning] {arc} | Γ=0: nominal={f_nom:.6f}, robust={sol['objective']:.6f} (both fractional)")

                        rows.append({
                            "File": arc,
                            "n": int(row["n"]),
                            "Capacity": cap,
                            "%Dev": pct_deviation,
                            "Gamma": gamma,
                            "%Gamma": pct,
                            "objective": sol["objective"],   # robust
                            "Eval c·x": sol["eval_cx"],      # nominal of the solution
                            "z": sol["z"],
                            "time_s": sol["time_s"],
                            "Status": sol["status"],
                            "n_vars": sol["n_vars"],
                            "n_constrs": sol["n_constrs"],
                        })
                    else:
                        print(f"  {arc} | Γ={gamma}: failure -> {sol['status']}")

    df_long_bs = pd.DataFrame(rows)
    if not df_long_bs.empty:
        df_long_bs[["objective", "Eval c·x", "z", "time_s"]] = df_long_bs[["objective", "Eval c·x", "z", "time_s"]].round(6)

    print("\n=== Summary (first rows) ===")
    print(df_long_bs.head(10))
    return df_long_bs


# ================== EXECUTION (example) ==================
df_long_bs = total_solver_bs_mip(
    df_kplib,
    pct_deviation_list=(0.1, 0.5, 1, 1.5, 2),
    gamma_pcts=(0.00, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00),
    sizes=(50, 100, 200, 500),
    k=20,
    time_limit=3600,
    check_nominal=True,
)


# ---------- BASELINE (Nominal = %Gamma == 0) and metric inc_vs_nominal ----------
nominal_bs = (
    df_long_bs[df_long_bs["%Gamma"] == 0][["File", "n", "Capacity", "objective"]]
    .drop_duplicates(subset=["File", "n", "Capacity"])
    .rename(columns={"objective": "Nominal"})
)

df_long_bs = df_long_bs.copy()
df_long_bs = df_long_bs.merge(nominal_bs, on=["File", "n", "Capacity"], how="left")

# Relative increase vs Nominal (lower-bounded at -100% if objective >= 0)
df_long_bs["inc_vs_nominal"] = (df_long_bs["objective"] - df_long_bs["Nominal"]) / df_long_bs["Nominal"]


# Aggregating Results

In [ ]:
def iqr(x):
    x = np.asarray(x, dtype=float)
    if x.size < 2:
        return np.nan
    return np.percentile(x, 75) - np.percentile(x, 25)

def ci95(x):
    """95% CI (low, high) for the mean using Student's t."""
    x = np.asarray(x, dtype=float)
    m = x.size
    if m < 2:
        return (np.nan, np.nan)
    mean = np.mean(x)
    se   = np.std(x, ddof=1) / np.sqrt(m)
    tcrit = student_t.ppf(0.975, df=m-1)
    h = tcrit * se
    return (mean - h, mean + h)




group_keys = ["n", "%Dev", "%Gamma"]

agg_bs_all = (
    df_long_bs
    .groupby(group_keys, dropna=False)
    .agg(
        count        = ("inc_vs_nominal", "size"),

        # increase vs Nominal
        inc_mean     = ("inc_vs_nominal", "mean"),
        inc_median   = ("inc_vs_nominal", "median"),
        inc_std      = ("inc_vs_nominal", "std"),
        inc_iqr      = ("inc_vs_nominal", iqr),
        inc_ci_low   = ("inc_vs_nominal", lambda x: ci95(x)[0]),
        inc_ci_high  = ("inc_vs_nominal", lambda x: ci95(x)[1]),

        # absolute objective
        obj_mean     = ("objective", "mean"),
        obj_median   = ("objective", "median"),
        obj_std      = ("objective", "std"),
        obj_iqr      = ("objective", iqr),
        obj_ci_low   = ("objective", lambda x: ci95(x)[0]),
        obj_ci_high  = ("objective", lambda x: ci95(x)[1]),

        # time
        time_mean    = ("time_s", "mean"),
        time_median  = ("time_s", "median"),
        time_std     = ("time_s", "std"),
        time_iqr     = ("time_s", iqr),
        time_ci_low  = ("time_s", lambda x: ci95(x)[0]),
        time_ci_high = ("time_s", lambda x: ci95(x)[1]),
    )
    .reset_index()
)

def classify_uncertainty(v):
    if v == 0.1:  return "Very low"
    if v == 0.5:  return "Low"
    if v == 1.0:  return "Medium"
    if v == 1.5:  return "High"
    if v == 2.0:  return "Very high"
    return "Unclassified"

agg_bs_all["Uncertainty"] = agg_bs_all["%Dev"].apply(classify_uncertainty)

for c in [col for col in agg_bs_all.columns if any(s in col for s in ("_mean","_median","_std","_iqr","_ci_low","_ci_high"))]:
    agg_bs_all[c] = agg_bs_all[c].astype(float).round(6)

cols_keys = ["n", "Uncertainty", "%Gamma"]
agg_bs_inc  = agg_bs_all[cols_keys + ["inc_mean","inc_ci_low","inc_ci_high","inc_std","inc_median","inc_iqr"]].copy()
agg_bs_obj  = agg_bs_all[cols_keys + ["obj_mean","obj_ci_low","obj_ci_high","obj_std","obj_median","obj_iqr"]].copy()
agg_bs_time = agg_bs_all[cols_keys + ["time_mean","time_ci_low","time_ci_high","time_std","time_median","time_iqr"]].copy()


print(agg_bs_inc["inc_mean"].value_counts())
display(agg_bs_inc)
display(agg_bs_obj)
display(agg_bs_time)


inc_mean
-1.000000    24
 0.000000    20
-0.100000     8
-0.500000     4
-0.767325     2
             ..
-0.487700     1
-0.173597     1
-0.323922     1
-0.644050     1
-0.410891     1
Name: count, Length: 79, dtype: int64


,n,Incertidumbre,%Gamma,inc_mean,inc_ci_low,inc_ci_high,inc_std,inc_median,inc_iqr
0,50,Muy Baja,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,50,Muy Baja,0.05,-0.009809,-0.010123,-0.009495,0.000672,-0.009805,0.000553
2,50,Muy Baja,0.10,-0.023680,-0.024347,-0.023013,0.001425,-0.023934,0.002018
3,50,Muy Baja,0.25,-0.052439,-0.053467,-0.051411,0.002197,-0.052878,0.002795
4,50,Muy Baja,0.50,-0.090421,-0.091357,-0.089485,0.002000,-0.091255,0.002719
...,...,...,...,...,...,...,...,...,...
135,500,Muy Alta,0.10,-0.410891,-0.413712,-0.408070,0.006028,-0.410239,0.005434
136,500,Muy Alta,0.25,-0.763157,-0.766909,-0.759405,0.008016,-0.764188,0.011313
137,500,Muy Alta,0.50,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000
138,500,Muy Alta,0.75,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000


,n,Incertidumbre,%Gamma,obj_mean,obj_ci_low,obj_ci_high,obj_std,obj_median,obj_iqr
0,50,Muy Baja,0.00,19955.588254,19275.794544,20635.381964,1452.505950,19841.211648,1616.378657
1,50,Muy Baja,0.05,19760.720680,19081.669927,20439.771434,1450.918485,19646.161648,1613.553657
2,50,Muy Baja,0.10,19484.842416,18808.886388,20160.798444,1444.306028,19369.061648,1595.978657
3,50,Muy Baja,0.25,18911.534444,18250.695891,19572.372996,1412.004723,18796.415318,1506.049374
4,50,Muy Baja,0.50,18151.735136,17529.085034,18774.385238,1330.407982,18038.134870,1437.189353
...,...,...,...,...,...,...,...,...,...
135,500,Muy Alta,0.10,119746.175580,118091.651966,121400.699195,3535.198045,119900.346590,3250.680992
136,500,Muy Alta,0.25,48147.996640,47100.339072,49195.654209,2238.515639,48206.931398,2230.104905
137,500,Muy Alta,0.50,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
138,500,Muy Alta,0.75,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,n,Incertidumbre,%Gamma,time_mean,time_ci_low,time_ci_high,time_std,time_median,time_iqr
0,50,Muy Baja,0.00,0.005566,0.003378,0.007754,0.004675,0.004696,0.001959
1,50,Muy Baja,0.05,0.003812,0.002291,0.005333,0.003249,0.002960,0.001694
2,50,Muy Baja,0.10,0.002581,0.002178,0.002984,0.000861,0.002901,0.001181
3,50,Muy Baja,0.25,0.003039,0.002093,0.003986,0.002023,0.002768,0.001256
4,50,Muy Baja,0.50,0.003407,0.002351,0.004462,0.002255,0.003100,0.001458
...,...,...,...,...,...,...,...,...,...
135,500,Muy Alta,0.10,0.009309,0.008258,0.010359,0.002245,0.008340,0.003747
136,500,Muy Alta,0.25,0.013192,0.011788,0.014595,0.002999,0.012331,0.005353
137,500,Muy Alta,0.50,0.009951,0.008916,0.010986,0.002212,0.008728,0.004070
138,500,Muy Alta,0.75,0.006282,0.005523,0.007041,0.001622,0.005637,0.002222


# Exporting Results

In [ ]:
agg_all_ro = agg_bs_all.copy()
order_inc = ["Very low", "Low", "Medium", "High", "Very high"]
agg_all_ro["Uncertainty"] = pd.Categorical(agg_all_ro["Uncertainty"], categories=order_inc, ordered=True)

# ====== 1) FIGURE X (RO): fix n=200; curves by Uncertainty; X=%Gamma; Y = inc_mean (+ 95% CI) ======
ro_fig_n200 = (
    agg_all_ro[agg_all_ro["n"] == 200]
    .loc[:, ["Uncertainty", "%Gamma", "inc_mean", "inc_ci_low", "inc_ci_high"]]
    .sort_values(["Uncertainty", "%Gamma"])
)

# (optional) percentage version with 2 decimals for Overleaf
ro_fig_n200_excel = ro_fig_n200.copy()
cols_pct = ["inc_mean", "inc_ci_low", "inc_ci_high"]
ro_fig_n200_excel.loc[:, cols_pct] = ro_fig_n200_excel.loc[:, cols_pct].applymap(lambda x: round(100 * float(x), 2))

# ====== 2) FIGURE Y (RO): fix Uncertainty = 'High'; grouped bars by n; series = %Gamma ======
ro_fig_uncer_high = (
    agg_all_ro[agg_all_ro["Uncertainty"] == "High"]
    .loc[:, ["n", "%Gamma", "inc_mean", "inc_ci_low", "inc_ci_high"]]
    .sort_values(["n", "%Gamma"])
)
ro_fig_uncer_high_excel = ro_fig_uncer_high.copy()
ro_fig_uncer_high_excel.loc[:, cols_pct] = ro_fig_uncer_high_excel.loc[:, cols_pct].applymap(lambda x: round(100 * float(x), 2))

# ====== 3) TIME FIGURE (RO): Uncertainty = 'High' and then average over all Uncertainty levels; time by n and %Gamma (mean + 95% CI) ======
ro_time_uncer_high = (
    agg_all_ro[agg_all_ro["Uncertainty"] == "High"]
    .loc[:, ["n", "%Gamma", "time_mean", "time_ci_low", "time_ci_high"]]
    .sort_values(["n", "%Gamma"])
)

ro_time_agg = (
    agg_all_ro
      .groupby(["n", "%Gamma"], as_index=False)
      .agg(time_mean_mean=("time_mean", "mean"),
           time_mean_std =("time_mean", "std"))
      .sort_values(["n", "%Gamma"])
)

# ====== 4) SUMMARY TABLES (RO): means by n, Uncertainty and %Gamma ======
table_mean_inc  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_mean"].mean()
table_mean_obj  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_mean"].mean()
table_mean_time = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_mean"].mean()

table_cilow_inc  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_ci_low"].mean()
table_cihigh_inc = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_ci_high"].mean()

table_cilow_time  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_ci_low"].mean()
table_cihigh_time = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_ci_high"].mean()

table_cilow_obj  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_ci_low"].mean()
table_cihigh_obj = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_ci_high"].mean()

table_std_inc  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_std"].mean()
table_std_time = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_std"].mean()
table_std_obj  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_std"].mean()

table_median_inc  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_median"].mean()
table_iqr_inc     = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["inc_iqr"].mean()
table_median_time = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_median"].mean()
table_iqr_time    = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["time_iqr"].mean()
table_median_obj  = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_median"].mean()
table_iqr_obj     = agg_all_ro.groupby(["n", "Uncertainty", "%Gamma"], as_index=False)["obj_iqr"].mean()

# ====== 5) EXPORT TO EXCEL ======
out_xlsx = "Experimentos Computacionales RO.xlsx"
with pd.ExcelWriter(out_xlsx) as writer:
    # Figure datasets
    ro_fig_n200_excel.to_excel(writer, sheet_name="FigX_n200_inc", index=False)
    ro_fig_uncer_high_excel.to_excel(writer, sheet_name="FigY_incAlta", index=False)
    ro_time_uncer_high.to_excel(writer, sheet_name="FigTiempo_incAlta", index=False)
    ro_time_agg.to_excel(writer, sheet_name="FigTiempo_agg_incert", index=False)

    # Full aggregates
    agg_all_ro.to_excel(writer, sheet_name="agg_all_ro", index=False)

    # Summary tables
    table_mean_inc.to_excel(writer, sheet_name="tabla_mean_inc", index=False)
    table_mean_obj.to_excel(writer, sheet_name="tabla_mean_obj", index=False)
    table_mean_time.to_excel(writer, sheet_name="tabla_mean_time", index=False)

    table_cilow_inc.to_excel(writer, sheet_name="tabla_cilow_inc", index=False)
    table_cihigh_inc.to_excel(writer, sheet_name="tabla_cihigh_inc", index=False)

    table_cilow_time.to_excel(writer, sheet_name="tabla_cilow_time", index=False)
    table_cihigh_time.to_excel(writer, sheet_name="tabla_cihigh_time", index=False)

    table_cilow_obj.to_excel(writer, sheet_name="tabla_cilow_obj", index=False)
    table_cihigh_obj.to_excel(writer, sheet_name="tabla_cihigh_obj", index=False)

    table_std_inc.to_excel(writer, sheet_name="tabla_std_inc", index=False)
    table_std_time.to_excel(writer, sheet_name="tabla_std_time", index=False)
    table_std_obj.to_excel(writer, sheet_name="tabla_std_obj", index=False)

    table_median_inc.to_excel(writer, sheet_name="tabla_median_inc", index=False)
    table_iqr_inc.to_excel(writer, sheet_name="tabla_iqr_inc", index=False)
    table_median_time.to_excel(writer, sheet_name="tabla_median_time", index=False)
    table_iqr_time.to_excel(writer, sheet_name="tabla_iqr_time", index=False)
    table_median_obj.to_excel(writer, sheet_name="tabla_median_obj", index=False)
    table_iqr_obj.to_excel(writer, sheet_name="tabla_iqr_obj", index=False)

# In Colab: download (comment out if not needed)
try:
    from google.colab import files
    files.download(out_xlsx)
except Exception:
    pass


/tmp/ipython-input-1738810498.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ro_fig_n200_excel.loc[:, cols_pct] = ro_fig_n200_excel.loc[:, cols_pct].applymap(lambda x: round(100 * float(x), 2))
/tmp/ipython-input-1738810498.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ro_fig_incer_alta_excel.loc[:, cols_pct] = ro_fig_incer_alta_excel.loc[:, cols_pct].applymap(lambda x: round(100 * float(x), 2))
/tmp/ipython-input-1738810498.py:52: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_mean_inc = agg_all_ro.groupby(["n", "Incertidumbre", "%Gamma"], as_index=False)["inc_mean"].mean()
/tmp/ipython-input-1738810498.py:53: FutureWarning: The default of observed=False is deprecated and will be changed to True in a fut

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>